# imports

In [4]:
import random

import numpy as np
from tqdm import tqdm

# open original vcf file

In [5]:
with open("/Users/minhtoo.lin/repos/nf-gwas/tests/input/pipeline/example.vcf", "r") as f:
    vcf_orig = f.readlines()

header_lines_orig = vcf_orig[:6]

# params

In [8]:
num_positions = 1_000_000
num_individuals = 50_000
# prev VCF (16 GB) was 100k positions x 100k individuals
# new VC (200 GB) is 1 million position x 50k individuals

# prepare new chromosome header line

In [6]:
# just copy the fixed part
chrom_header_line_base = "#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT"

chrom_header_line_chars = [chrom_header_line_base]
for i in range(1, num_individuals + 1):  # 1-indexed
    chrom_header_line_chars.append(f"\t{i}")
chrom_header_line_chars.append("\n")

chrom_header_line_new = "".join(chrom_header_line_chars)

with open("../data/example_bigger.vcf", "w") as f:
    # write 6 header lines
    f.writelines(header_lines_orig)
    # write new chromosome header line
    f.write(chrom_header_line_new)

# prepare new data lines, row by row - use random.choice()

In [8]:
GT_CHOICES = np.array(["0/0", "0/1", "1/1"])

# append
with open("../data/example_bigger.vcf", "a") as f:
    for row_idx in tqdm(range(1, num_positions + 1)):  # 1-indexed, up to num_position rows
        line_base = f"1\t{row_idx}\t{row_idx}\t2\t1\t.\t.\tPR\tGT"
        curr_line_chars = [line_base]
        
        randints = np.random.randint(0, len(GT_CHOICES), size=(num_individuals))
        curr_line_chars.append("\t")
        curr_line_chars.append("\t".join(GT_CHOICES[randints]))
        curr_line_chars.append("\n")

        f.write("".join(curr_line_chars))

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1000000/1000000 [3:15:43<00:00, 85.16it/s]


# write fake phenotypes file

In [11]:
num_phenotypes_total = 20
num_phenotypes_per_file = 20

assert num_phenotypes_total % num_phenotypes_per_file == 0

header_base = "FID IID"
curr_pheno_idx = 1  # 1-indexed
for file_idx in range(num_phenotypes_total // num_phenotypes_per_file):
    with open(f"../data/phenotype_biggest_20_{file_idx}.txt", "w") as f:
        # write header
        header_chars = [header_base]
        for pheno_idx in range(curr_pheno_idx, curr_pheno_idx + num_phenotypes_per_file):
            header_chars.append(f" Y{pheno_idx}")
        header_chars.append("\n")
        f.write("".join(header_chars))

        # draw from normal distribution
        random_nums = np.random.normal(size=(num_individuals, num_phenotypes_per_file))

        # write data rows
        for row_idx in tqdm(range(1, num_individuals + 1)): # 1-indexed
            row_chars = [f"{row_idx} {row_idx}"]
            for pheno_idx in range(num_phenotypes_per_file):
                row_chars.append(f" {random_nums[row_idx - 1][pheno_idx]}")  # 0-indexed
            row_chars.append("\n")
            f.write("".join(row_chars))

    curr_pheno_idx += num_phenotypes_per_file

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 71659.92it/s]


# write rsids.tsv

In [10]:
rsid_init = 270000
header_line = "CHROM	POS	RSID	REF	ALT	AAF\n"
with open("../data/rsids_big.tsv", "w") as f:
    f.write(header_line)

    for row_idx in tqdm(range(1, num_positions + 1)):
        f.write(f"1\t{row_idx}\trs{rsid_init + row_idx}\t2\t1\t0,1\n")

100%|█████████████████████████████████████████████████████████████████████████████████████████| 1000000/1000000 [00:00<00:00, 2727910.34it/s]
